In [40]:
import numpy as np
import theano
import theano.tensor as T

import lasagne as L

In [41]:
def build_simple_rnnlm(input_var, mask_input_var, voc_size, emb_size, rec_size):
    l_in = L.layers.InputLayer(shape=(None, None), input_var=input_var)    
    batch_size, seq_len = l_in.input_var.shape
    l_mask = L.layers.InputLayer(shape=(batch_size, seq_len), input_var=mask_input_var)
    
    l_emb = L.layers.EmbeddingLayer(l_in,
                                    input_size=voc_size, 
                                    output_size=emb_size)
    
    l_rec = L.layers.RecurrentLayer(l_emb,
                                    num_units=rec_size, 
                                    W_in_to_hid=L.init.Orthogonal(), 
                                    W_hid_to_hid=L.init.Orthogonal(),
                                    mask_input=l_mask)
    
    l_resh = L.layers.ReshapeLayer(l_rec, shape=(-1, rec_size))
    
    l_soft = L.layers.DenseLayer(l_resh,
                                num_units=voc_size,
                                nonlinearity=L.nonlinearities.softmax)
    
    l_out = L.layers.ReshapeLayer(l_soft, shape=(batch_size, seq_len, voc_size))
    
    return l_out

In [45]:
voc_size = 10
emb_size = 50
rec_size = 100

input_var = T.imatrix('inputs')
mask_input_var = T.matrix('input_mask')

net = build_simple_rnnlm(input_var, mask_input_var, voc_size, emb_size, rec_size)
out = L.layers.get_output(net)

targets = input_var # we want to maximize the prob. of generating a train sequence

loss = L.objectives.categorical_crossentropy(out.reshape((-1,voc_size)), 
                                             targets.reshape(targets.shape, ndim=1))
loss = loss.mean() # mean batch loss

params = L.layers.get_all_params(net, trainable=True)
updates = L.updates.rmsprop(loss, params)

train_fn = theano.function([input_var, mask_input_var], loss, updates=updates)

#### for validation

test_out = L.layers.get_output(net, deterministic=True)
test_loss = L.objectives.categorical_crossentropy(test_out.reshape((-1,voc_size)), 
                                                  targets.reshape(targets.shape, ndim=1))
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(T.argmax(test_out, axis=1), targets), dtype=theano.config.floatX)

val_fn = theano.function([input_var, mask_input_var], [test_loss, test_acc])